In [2]:
from pyspark.ml.feature import StringIndexer
from pyspark import SparkContext
from pyspark.sql import SparkSession
import sys
import getpass


def main(spark):
           
    # read in data
    trainSample = spark.read.parquet('../data/train_sample1pct.parquet')
    
    trainSample.createOrReplaceTempView('trainSample')

#     # StringIndexer to create new columns and dataframes
    indexer_obj_1 = StringIndexer(inputCol="user_id", outputCol="user_id_numer").setHandleInvalid("keep")
    indexer_model_1 = indexer_obj_1.fit(trainSample)
    indexer_df_1 = indexer_model_1.transform(trainSample)
    print('finished indexer 1 on train')

    indexer_obj_2 = StringIndexer(inputCol="track_id", outputCol="track_id_numer").setHandleInvalid("keep")
    indexer_model_2 = indexer_obj_2.fit(indexer_df_1)
    indexer_df_2 = indexer_model_2.transform(indexer_df_1)
    print('finished indexer 2 on train')

    train_df = indexer_df_2.drop('user_id')
    train_df = train_df.drop('track_id')
    train_df = train_df.repartition(2000)
    
    print('dropped columns in training set')

    train_df.write.mode('overwrite').parquet('../data/train_df_1pct.parquet')


if __name__ == "__main__":
        # Create the spark session object
    spark = SparkSession.builder.config('spark.driver.memory', '16g').appName('sampler').getOrCreate()
    
    main(spark)

finished indexer 1 on train
finished indexer 2 on train
dropped columns in training set
